In [330]:
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np
import cv2
from dataclasses import dataclass
from pyglet.window import key
from scipy.spatial import ConvexHull
import time
from statistics import mean

In [331]:
distances = []

In [332]:
class CentroidTracker():
    def __init__(self, maxDisappeared=50):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.areas = OrderedDict()
        self.disappeared = OrderedDict()
        self.memory = OrderedDict()
        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared
        self.totalIn = [0]*len(zones)
        self.totalOut = [0]*len(zones)
        
    def register(self, centroid, area):
        # when registering an object we use the next available object
        # ID to store the centroid
        tmp = 0
        for i, zona in enumerate(zones):
            if isinside(creation_in_out_spaces(zona), (centroid[0], centroid[1])):
                self.totalIn[i] += 1
                tmp += 1
        if tmp > 0: print('There is one NEW appeared in in the middle of the frame --> ID: ', self.nextObjectID)
        self.objects[self.nextObjectID] = centroid
        self.areas[self.nextObjectID] = area
        self.memory[self.nextObjectID] = [list(centroid)]
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1
        
    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        tmp = 0
        for i, zona in enumerate(zones):
            if isinside(creation_in_out_spaces(zona), (objects[objectID][0], objects[objectID][1])):
                self.totalOut[i] += 1
                tmp += 1
        if tmp > 0: print('This detection has desappeared outside the ROIs ---> ID: ', objectID)
        del self.objects[objectID]
        del self.areas[objectID]
        del self.disappeared[objectID]

    def update(self, inputCentroids, inputAreas):
        # check to see if the list of input bounding box rectangles
        # is empty
        if len(inputCentroids) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            # return early as there are no centroids or tracking info
            # to update
            return self.objects, self.memory   
        # if we are currently not tracking any objects take the input
        # centroids and register each of them
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                #self.register(inputCentroids[i], areas[i])
                self.register(inputCentroids[i])
        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())
            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            #D = np.where(D > 20, 0, an_array)
            objectAreas = list(self.areas.values())
            A = []
            for objectArea in objectAreas:
                sub = [abs(area - objectArea) for area in inputAreas]
                A.append(sub)
            A = np.array(A)
            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value is at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()
            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]
            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()
            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                # val
                if row in usedRows or col in usedCols:
                    continue
                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.memory[objectID].append(list(inputCentroids[col]))
                self.disappeared[objectID] = 0
                # indicate that we have examined each of the row and
                # column indexes, respectively
                distances.append(D[row][col])
                usedRows.add(row)
                usedCols.add(col)
            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)
            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1
                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
                        
            # otherwise, if the number of input centroids is greater
            # than the number of existing object centroids we need to
            # register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    #self.register(inputCentroids[col], areas[col])
                    self.register(inputCentroids[col])
        # return the set of trackable objects
        return self.objects, self.memory

In [333]:
@dataclass
class Point:
    x: int
    y: int
        
@dataclass
class zone:
    start: Point
    end: Point
    extense: int
    its_h: bool

In [334]:
def creation_in_out_spaces(zone):
    #de momento los horizontales son igual de amplios por dentro y por fuera
    start = [zone.start.x, zone.start.y]
    end = [zone.end.x, zone.end.y]
    extense = zone.extense

    if zone.its_h:
        zone_space = [[start[0] + extense, start[1]],
                         [start[0], start[1]],
                         [end[0], end[1]],
                         [end[0] + extense, end[1]]]

    #esta hecho para que la tienda mire hacia abajo y no hacia arriba
    else:
        zone_space = [[start[0], start[1] - extense],
                         [end[0], end[1] - extense],
                         [end[0], end[1]+20],
                         [start[0], start[1]+20]]

    return zone_space

def isinside(l, p):
    poly = np.array(l,dtype=np.int32)
    poly_new = poly.reshape((-1,1,2))
    result1 = cv2.pointPolygonTest(poly_new, p, False)
    #cv2.polylines(medianFrame,[poly_new],isClosed=True,color=(0,255,0),thickness=10)
    #plt.imshow(medianFrame)
    if result1 == 1.0:
        return True
    else:
        return False
def creation_in_out_spaces(zone):
    #de momento los horizontales son igual de amplios por dentro y por fuera
    start = [zone.start.x, zone.start.y]
    end = [zone.end.x, zone.end.y]
    extense = zone.extense

    if zone.its_h:
        zone_space = [[start[0] + extense, start[1]],
                         [start[0], start[1]],
                         [end[0], end[1]],
                         [end[0] + extense, end[1]]]

    #esta hecho para que la tienda mire hacia abajo y no hacia arriba
    else:
        zone_space = [[start[0], start[1] - extense],
                         [end[0], end[1] - extense],
                         [end[0], end[1]+20],
                         [start[0], start[1]+20]]

    return zone_space

In [335]:
l_ofpoints = [[0, 0], [0, 300], [960, 350], [960, 0]]
l_ofpoints2 = [[0, 500], [0, 720], [960, 720], [960, 600]]
lists_out = [l_ofpoints, l_ofpoints2]

In [336]:
width = 960
height = 720
zona0 = zone(Point(width/9, 300), Point(width/9, 600), -width/9, True)
zona1 = zone(Point((8/9)*width, 295), Point((8/9)*width, 650), width/9, True)
zona2 = zone(Point(780, 450), Point(860, 450), 60, False)
zones = [zona0, zona1, zona2]

In [337]:
video_path = '../PycharmProjects/pythonProject/venv/COUNTING/Videos/23-02-2021.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
ct = CentroidTracker()
out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), fps, (width, height))
prova = []
while cap.isOpened:
    ret, img = cap.read()
    if ret == True:
        img = cv2.resize(img, (width,height))
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        fgmask = fgbg.apply(img)
        kernel = np.ones((2,2), np.uint8)
        fgmask = cv2.erode(fgmask, kernel, iterations=1)
        fgmask = cv2.dilate(fgmask, kernel, iterations=1)
        cv2.imshow('mask', fgmask)
        cv2.imwrite('mask.jpg', fgmask)
        (contours, hierarchy) = cv2.findContours(fgmask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        centroids = []
        areas = []
        for index, contour in enumerate(contours):
            #(x, y, w, h) = cv2.boundingRect(contour)
            #(startX, startY, endX, endY) = (x, y, x+w, y+h)
            #center = (int(startX+(w/2)), int(startY+(h/2)))
            hull = cv2.convexHull(contour)
            area = cv2.contourArea(hull)
            #if w>40 and h>70:
            if area<8207 and area>1640:
                hull_ = hull[:, 0, :]
                hull_ = ConvexHull(hull_)
                centroid = (np.mean(hull_.points[hull_.vertices,0]), np.mean(hull_.points[hull_.vertices,1]))
                if any([isinside(list, centroid) for list in lists_out]):
                    continue
                #cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 1)
                #rects.append((x, y, x+h, y+h))
                centroids.append(centroid)
                cv2.polylines(img,[hull],True,(0,255,255), 2)
                areas.append(area)
                prova.append(area)
        assert len(centroids) == len(areas)
        objects, memory = ct.update(centroids, areas)
        for (objectID, centroid) in objects.items():
            # draw both the ID of the object and the centroid of the
            # object on the output frame
            text = "ID {}".format(objectID)
            cv2.putText(img, text, (int(centroid[0]) - 10, int(centroid[1]) - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)       
            cv2.circle(img, (int(centroid[0]), int(centroid[1])), 4, (0, 255, 0), -1)

        txt = 'Total Outs: {}'
        #IZQUIERDA
        cv2.rectangle(img, (0, height-(2*40)), (len(txt)*13, height), (255,255,255), thickness=-1)
        cv2.putText(img, 'Total Ins: {}'.format(ct.totalIn[0]), (20, int(height)-20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(img, 'Total Outs: {}'.format(ct.totalOut[0]), (20, int(height)-50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        #DERECHA
        cv2.rectangle(img, (width-len(txt)*13, height-(2*40)), (width, height), (255,255,255), thickness=-1)
        cv2.putText(img, 'Total Ins: {}'.format(ct.totalIn[1]), (width-len(txt)*11, int(height)-20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(img, 'Total Outs: {}'.format(ct.totalOut[1]), (width-len(txt)*11, int(height)-50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        #TIENDA
        cv2.rectangle(img, (zones[2].start.x-20,  zones[2].start.y-zones[2].extense-70), (zones[2].start.x+len(txt)*12, zones[2].start.y-zones[2].extense-10), (255,255,255), thickness=-1)
        cv2.putText(img, 'Total Outs: {}'.format(ct.totalIn[2]), (zones[2].start.x, zones[2].start.y-zones[2].extense-20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(img, 'Total Ins: {}'.format(ct.totalOut[2]), (zones[2].start.x, zones[2].start.y-zones[2].extense-50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        cv2.imshow('frame', img)
        out.write(img)
        if cv2.waitKey(1) & 0xFF == key.SPACE:
            break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

30.0


TypeError: register() missing 1 required positional argument: 'area'

In [329]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Box(y=prova)) #areas
fig.show()

In [289]:
count = len([elem for elem in areas if elem < 8936])
count

2

In [246]:
print(type(hull))
#Get centoid
cx = np.mean(hull.points[hull.vertices,0])
cy = np.mean(hull.points[hull.vertices,1])
cx, cy

<class 'scipy.spatial.qhull.ConvexHull'>


(716.0, 148.0)

In [244]:
hull = hull[:, 0, :]
hull = ConvexHull(hull)

In [245]:
hull.points

array([[716., 145.],
       [717., 147.],
       [717., 149.],
       [716., 151.],
       [715., 151.],
       [715., 145.]])

In [218]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Box(y=distances))
fig.show()